In [ ]:
!pip install -e MeMoKBC/

In [ ]:
!python bio-medrxiv.py <db-name> -f /data/Goldlabel_biomedRxiv/goldlabel1_docs801-840_laura/goldlabel1_tasknameshort_MedRxiv_docs/ -c -M -C -L

imports

In [45]:
from fonduer.supervision import Labeler
from fonduer.supervision.models import GoldLabel
from fonduer.features import Featurizer
from fonduer.candidates.models import Candidate

from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import LabelModel

from MeMoKBC.pipeline.utils import get_session, load_candidates, match_label_matrix
from MeMoKBC.definitions.candidates import NameFullAbbr, NameAbbrTask
from MeMoKBC.pipeline.lfs.name_short_long_lfs import short_long_lfs
from MeMoKBC.pipeline.lfs.name_short_task_lfs import name_abbr_task_lfs
from MeMoKBC.gold_label_matcher import match_gold_label

Get session object

In [ ]:
session = get_session(db_name="<db-name>")

Define candidates and Labeler object

In [ ]:
candidates = [NameAbbrTask, NameFullAbbr]


Load candidates and labels

In [ ]:
L_train_NAT, L_train_NFA = match_label_matrix(session, candidates, 0)

## Load gold labels

In [ ]:
gold_labels = match_gold_label(
    "<db-name>",
    "/data/Goldlabel_biomedRxiv/goldlabel1_docs801-840_laura/goldlabel_authorlong_short_task_medRxiv.json",
    [NameAbbrTask, NameFullAbbr]
)

nat_cands = []
nfa_cands = []
for cand in gold_labels:
    if type(cand) == NameAbbrTask:
        if cand[0].context.sentence.id == cand[1].context.sentence.id:
            nat_cands.append(cand.id)
    elif type(cand) == NameFullAbbr:
        nfa_cands.append(cand.id)

In [ ]:
labeler = Labeler(session, candidates)

In [ ]:
def gold(c: Candidate) -> int:
    if type(c) == NameAbbrTask:
        if c.id in nat_cands:
            return 1

    elif type(c) == NameFullAbbr:
        if c.id in nfa_cands:
            return 1

    return 0

In [ ]:
labeler.apply(lfs=[[gold], [gold]], table=GoldLabel, train=True)

In [ ]:
train_cands = load_candidates(session, 0, candidates)

In [ ]:
L_gold_train_NAT, L_gold_train_NFA = labeler.get_gold_labels(train_cands, annotator="gold")

## LF analysis

NameFull + Abrreviation

In [ ]:
LFAnalysis(
    L_train_NFA,
    lfs=sorted(short_long_lfs, key=lambda lf: lf.name)
).lf_summary(Y=L_gold_train_NFA.reshape(-1))

NameAbbr + Task

In [ ]:
LFAnalysis(
    L_train_NAT,
    lfs=sorted(name_abbr_task_lfs, key=lambda lf: lf.name)
).lf_summary(Y=L_gold_train_NAT.reshape(-1))